In [ ]:
# 1) (a) 0.01
# (b) The weight would be larger to give more focus on the observation that was misclassified.

In [ ]:
# 2) AdaBoost is an ensemble learning because it continously fixes the next model to better itself.

In [ ]:
# 3) 0.16 -> 0.06,  0.64 -> 0.54,  0.08 -> 0.18,  0.12 -> 0.22

In [ ]:
# 4) Would increase the depth of the decision tree as it will better the model in its estimating.

In [ ]:
# 5) C

In [ ]:
# 6) F

In [ ]:
# 7) AdaBoost is preformed to increase the weights of the weaker learners while Gradient boosting 
# finds observations with high residuals.

In [1]:
# 8)
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

s3 = boto3.resource('s3')
bucket_name = 'data-445-timlincoln'
bucket = s3.Bucket(bucket_name)

file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the data-file
heart = pd.read_csv(file_content_stream)

# Dropping N/A variables
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
# Defining the input and target variables
X = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

# Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [3]:
# Building the random forest model
RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

# Predicting on test
RF_pred = RF_md.predict_proba(X_test)[:, 1]

# Changing to labels
RF_labels = np.where(RF_pred < 0.1, 0, 1)

print('The accuracy of the random forest model is:', accuracy_score(Y_test, RF_labels))
print('The recall of the random forest model is:', recall_score(Y_test, RF_labels))

The accuracy of the random forest model is: 0.46994535519125685
The recall of the random forest model is: 0.8080808080808081


In [4]:
# Building the ada boost model
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)

# Predicting on test
ada_pred = ada_md.predict_proba(X_test)[:, 1]

# Changing to labels
ada_labels = np.where(ada_pred < 0.1, 0, 1)

print('The accuracy of the ada boost model is:', accuracy_score(Y_test, ada_labels))
print('The recall of the ada boost model is:', recall_score(Y_test, ada_labels))

The accuracy of the ada boost model is: 0.13797814207650272
The recall of the ada boost model is: 1.0


In [5]:
# Building the gradient boosting model
gb_md = GradientBoostingClassifier(max_depth = 3, n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)

# Predicting on test
gb_pred = gb_md.predict_proba(X_test)[:, 1]

# Changing to labels
gb_labels = np.where(gb_pred < 0.1, 0, 1)

print('The accuracy of the ada boost model is:', accuracy_score(Y_test, gb_labels))
print('The recall of the Gradient boosting model is:', recall_score(Y_test, gb_labels))

The accuracy of the ada boost model is: 0.5013661202185792
The recall of the Gradient boosting model is: 0.7474747474747475


In [8]:
# Creating list for average
RF_acc = []
ada_acc = []
gb_acc = []

# Creating list for recall
RF_rec = []
ada_rec = []
gb_rec = []

In [10]:
for i in range(0, 100):
    # Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
    
    # Building the random forest model
    RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

    # Predicting on test
    RF_pred = RF_md.predict_proba(X_test)[:, 1]

    # Changing to labels
    RF_labels = np.where(RF_pred < 0.1, 0, 1)
    
    # Appending accuarcy and recall to list
    RF_acc.append(accuracy_score(Y_test, RF_labels))
    RF_rec.append(recall_score(Y_test, RF_labels))
    
# Computing average
print('The average accuracy score is:', np.mean(RF_acc))
print('The average recall score is:', np.mean(RF_rec))

The average accuracy score is: 0.4500281897822881
The average recall score is: 0.8489405950918391


In [11]:
for n in range(0, 100):
    # Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
    
    # Building the ada boost model
    ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)

    # Predicting on test
    ada_pred = ada_md.predict_proba(X_test)[:, 1]

    # Changing to labels
    ada_labels = np.where(ada_pred < 0.1, 0, 1)
    
    # Appending accuarcy and recall to list
    ada_acc.append(accuracy_score(Y_test, ada_labels))
    ada_rec.append(recall_score(Y_test, ada_labels))
    
# Computing average
print('The average accuracy score is:', np.mean(ada_acc))
print('The average recall score is:', np.mean(ada_rec))

The average accuracy score is: 0.1594262295081967
The average recall score is: 0.9908356613741874


In [12]:
for n in range(0, 100):
    # Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
    
    # Building the gradient boosting model
    gb_md = GradientBoostingClassifier(max_depth = 3, n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)

    # Predicting on test
    gb_pred = gb_md.predict_proba(X_test)[:, 1]

    # Changing to labels
    gb_labels = np.where(gb_pred < 0.1, 0, 1)
    
    # Appending accuarcy and recall to list
    gb_acc.append(accuracy_score(Y_test, gb_labels))
    gb_rec.append(recall_score(Y_test, gb_labels))
    
# Computing average
print('The average accuracy score is:', np.mean(gb_acc))
print('The average recall score is:', np.mean(gb_rec))

The average accuracy score is: 0.5049180327868852
The average recall score is: 0.8043423601643468


In [ ]:
# From the three models built, the best model to predict TenYearCHD for accuracy and almost the best for recall 
# is gradient boosting with an average accuracy score of 0.50 and an average recall score of 0.80.

In [ ]:
# c) With the accuracy score of the gradient boosting model only being 0.50, the model does not meet the minimum
# reqiurements and would need fixing. What I would recommend is to decrease the learning rate so that maybe
# the accuracy would increase on the prediction.